In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Using TensorFlow backend.


In [2]:
IMAGE_SIZE = [224, 224]

In [3]:
#Give dataset path
train_path = '/content/drive/My Drive/FamilyClassifierData'
test_path = '/content/drive/My Drive/FamilyClassifierData'

In [4]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [6]:
# useful for getting number of classes
folders = glob('/content/drive/My Drive/FamilyClassifierData/*')
print(len(folders))

9


In [7]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [8]:
from keras import optimizers


sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [9]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [10]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [11]:
train_set = train_datagen.flow_from_directory('/content/drive/My Drive/FamilyClassifierData',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 111 images belonging to 9 classes.


In [27]:
train_set.class_indices

{'Alexa': 0,
 'Anup Varadkar': 1,
 'Mug': 2,
 'Papa': 3,
 'Pradip Shirke': 4,
 'Rahul Nandanwar': 5,
 'Shubham Nandanwar': 6,
 'Tejas Patil': 7,
 'mummy': 8}

In [12]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/FamilyClassifierData',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 111 images belonging to 9 classes.


In [13]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

#lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

#num_epochs = 1000
#num_batch_size = 32

checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=1, save_best_only=True)

#callbacks = [checkpoint, lr_reducer, lr_scheduler]
callbacks = [checkpoint, lr_reducer]

start = datetime.now()

model.fit_generator(train_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=10,
  validation_steps=32,
  callbacks=callbacks ,verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/25
10/10 [==============================] - 734s 73s/step - loss: 617.7138 - accuracy: 0.4615 - val_loss: 101.3983 - val_accuracy: 0.7973

Epoch 00001: val_loss improved from inf to 101.39830, saving model to mymodel.h5
Epoch 2/25
10/10 [==============================] - 717s 72s/step - loss: 269.6961 - accuracy: 0.8141 - val_loss: 50.2150 - val_accuracy: 0.8874

Epoch 00002: val_loss improved from 101.39830 to 50.21503, saving model to mymodel.h5
Epoch 3/25
10/10 [==============================] - 718s 72s/step - loss: 116.2597 - accuracy: 0.8951 - val_loss: 119.2791 - val_accuracy: 0.9155

Epoch 00003: val_loss did not improve from 50.21503
Epoch 4/25
10/10 [==============================] - 718s 72s/step - loss: 94.6332 - accuracy: 0.9331 - val_loss: 2.2178 - val_accuracy: 0.9099

Epoch 00004: val_loss improved from 50.21503 to 2.21776, saving model to mymodel.h5
Epoch 5/25
10/10 [==============================] - 714s 71s/step - loss: 79.6591 - accuracy: 0.9336 - val_loss: 

In [37]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img(r'/content/drive/My Drive/FamilyClassifierData/Mug/IMG_20200530_230438.jpg', target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

In [38]:
result = model.predict(test_image)
result = np.argmax(result)

In [39]:
result

2